In [1]:
import pandas as pd
import numpy as np
from pyscipopt import Model, quicksum, multidict
import os
os.chdir('/Users/user/PycharmProjects/Shell_hachakton_2023')

In [2]:
data = pd.read_csv('data/Biomass_History.csv', index_col=0)
dist = pd.read_csv('data/Distance_Matrix.csv', index_col=0)
biomass_2010 = data['2010']

In [3]:
def flp(I,J,d,M,f,c):
    model = Model("flp")
    x,y = {},{}
    for j in J:
        y[j] = model.addVar(vtype="B", name="y(%s)"%j)
        for i in I:
            x[i,j] = model.addVar(vtype="C", name="x(%s,%s)"%(i,j))
    for i in I:
        model.addCons(quicksum(x[i,j] for j in J) == d[i], "Demand(%s)"%i)
    for j in M:
        model.addCons(quicksum(x[i,j] for i in I) <= M[j]*y[j], "Capacity(%s)"%i)
    for (i,j) in x:
        model.addCons(x[i,j] <= d[i]*y[j], "Strong(%s,%s)"%(i,j))
    model.setObjective(
        quicksum(f[j]*y[j] for j in J) +
        quicksum(c[i,j]*x[i,j] for i in I for j in J),
        "minimize")
    model.data = x,y
    return model
    
I, d = multidict({1:80, 2:270, 3:250, 4:160, 5:180})
J, M, f = multidict({1:[500,1000], 2:[500,1000], 3:[500,1000]})
c = {(1,1):4,  (1,2):6,  (1,3):9,
      (2,1):5,  (2,2):4,  (2,3):7,
      (3,1):6,  (3,2):3,  (3,3):4,
      (4,1):8,  (4,2):5,  (4,3):3,
      (5,1):10, (5,2):8,  (5,3):4,
      }

In [5]:
model = flp(I, J, d, M, f, c)
model.optimize()
EPS = 1.e-6
x,y = model.data
edges = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]
facilities = [j for j in y if model.getVal(y[j]) > EPS]
print("Optimal value=", model.getObjVal())
print("Facilities at nodes:", facilities)
print("Edges:", edges)

Optimal value= 5610.0
Facilities at nodes: [2, 3]
Edges: [(1, 2), (2, 2), (3, 2), (3, 3), (4, 3), (5, 3)]


In [6]:
model.getVal(x[2,2])

270.0

In [81]:
def CFLP(I,J,d,M,c):
    model = Model("flp")
    x,y = {},{}
    for j in J:
        y[j] = model.addVar(vtype="B", name="y(%s)"%j)
        for i in I:
            x[i,j] = model.addVar(vtype="C", name="x(%s,%s)"%(i,j))
    for i in I:
        model.addCons(quicksum(x[i,j] for j in J) == d[i], "Demand(%s)"%i) # All demands should be met
    for j in M:
        model.addCons(quicksum(x[i,j] for i in I) <= M[j]*y[j], "Capacity(%s)"%i) # All facility should not violate capacity constraint
    for (i,j) in x:
        model.addCons(x[i,j] <= d[i]*y[j], "Strong(%s,%s)"%(i,j)) # Can't be delivered more than demand

    model.setObjective(
        quicksum((M[j]*y[j] - quicksum(x[i,j] for i in I)) for j in J) +
        0.001 * quicksum(c[i,j]*x[i,j] for i in I for j in J),
        "minimize")
    model.data = x,y
    return model

I, d = multidict(dict(biomass_2010[0:10]))
capacity = pd.Series(20000, index=biomass_2010[0:10].index)
J, M = multidict(dict(capacity))
c = dist.iloc[0:10,0:10].reset_index().melt(id_vars='index')
c['variable'] = c['variable'].astype('int')
c = {(row[0], row[1]):row[2] for row in c.values}

In [82]:
model = CFLP(I, J, d, M, c)
model.optimize()
EPS = 1.e-6
x,y = model.data
edges = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]
facilities = [j for j in y if model.getVal(y[j]) > EPS]
print("Optimal value=", model.getObjVal())
print("Facilities at nodes:", facilities)
print("Edges:", edges)

Optimal value= 31714.587552468915
Facilities at nodes: [5]
Edges: [(0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5), (6, 5), (7, 5), (8, 5), (9, 5)]


In [85]:
model.getVal(x[2,5])

44.83163452

In [10]:
year = 2018
data_type = 'biomass_demand_supply'

def get_routes_solution(model, edges, year, data_type):
    solution = np.zeros(shape=(len(edges), 3))
    for i,edge in enumerate(edges):
        solution[i,0] = edge[0]
        solution[i,1] = edge[1]
        solution[i,2] = model.getVal(x[edge])
    solution = pd.DataFrame(solution, columns=['source_index', 'destination_index', 'value'])
    solution['year'] = year
    solution['data_type'] = data_type
    solution['source_index'] = solution['source_index'].astype('int')
    solution['destination_index'] = solution['destination_index'].astype('int')
    solution = solution[['year', 'data_type', 'source_index', 'destination_index', 'value']]
    
    return solution

def get_locations(model, facilities, year, data_type):

    solution = pd.DataFrame(np.zeros(shape=(len(facilities), 5)), columns = ['year', 'data_type', 'source_index', 'destination_index', 'value'])
    solution['year'] = year; solution['data_type'] = data_type
    solution['destination_index'] = np.nan; solution['value'] = np.nan
    solution['source_index'] = facilities

    return solution

In [ ]:
get_routes_solution(model, edges, year, data_type)

In [11]:
get_locations(model, facilities, year, data_type)

,year,data_type,source_index,destination_index,value
0,2018,biomass_demand_supply,2,NaN,NaN
1,2018,biomass_demand_supply,3,NaN,NaN
